In [31]:
import os, subprocess, warnings
from datetime import datetime
from tabulate import tabulate

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from datetime import datetime
from scipy.integrate import quad, quad_vec, trapz, cumtrapz
from scipy.optimize import fsolve
from scipy.ndimage import gaussian_filter
from scipy.special import erf
from scipy.interpolate import griddata, interpn
from scipy.signal import welch

from functools import cached_property

from sim import *
import const

import yt


In [32]:
sim_idx = GAS


In [33]:
sim_name = list_of_sim_name[sim_idx]
sim_latex = list_of_sim_latex[sim_idx]
sim_dir = os.path.join(sim_base_dir, sim_name)

os.chdir(sim_dir)


In [34]:
def get_stdout(cmd):
    ''' Return the standard output of a command line directive '''
    stdout = subprocess.check_output(cmd, shell=True).decode()
    return stdout


In [35]:
class Halo(object):
    '''
    Object for storing data from halo files.
    
    Args
    stdout_split: standard output of a halo search command
    
    Attrs
    idx (int): halo index
    ncell (int): number of dark matter particles in the halo
    coord: coordinates of the halo (in code units before running convert_to_cgs)
    rho_max (float): max density in the halo in code units (in code units before running convert_to_cgs)
    mass (float): mass of the halo in code units (in code units before running convert_to_cgs)
    '''
    def __init__(self, stdout_split):
        
        self.idx = int(stdout_split[0])
        self.ncell = int(stdout_split[1])
        self.coord = np.array(stdout_split[2:5], dtype=float)
        self.rho_max = float(stdout_split[5])
        self.mass = float(stdout_split[6])
        
    def convert_to_cgs(self):
        ''' Convert attributes to cgs units. '''
        self.coord *= length_unit
        self.rho_max *= density_unit
        self.mass *= mass_unit
        
        
class Clump(object):
    '''
    Object for storing data from clump files.
    
    Args
    stdout_split: standard output of a clump search command
    
    Attrs
    idx (int): clump index
    lev (int): number of merging iterations away from halo
    parent_idx (int): index of the clump one level below
    ncell (int): number of dark matter particles in the clump
    coord: coordinates of the clump (in code units before running convert_to_cgs)
    rho_border (float): max density on the clump boundary (in code units before running convert_to_cgs)
    rho_max (float): max density in the clump (in code units before running convert_to_cgs)
    mass (float): mass of the clump (in code units before running convert_to_cgs)
    '''
    def __init__(self, stdout_split):
        
        stdout_split = stdout.split()
        
        self.idx = int(stdout_split[0])
        self.level = int(stdout_split[1])
        self.parent_idx = int(stdout_split[2])
        self.ncell = int(stdout_split[3])
        self.coord = np.array(stdout_split[4:7], dtype=float)
        self.rho_border = float(stdout_split[7])
        self.rho_max = float(stdout_split[8])
        self.mass = float(stdout_split[9])
        
    def convert_to_cgs(self):
        ''' Convert attributes to cgs units. '''
        self.coord *= length_unit
        self.rho_border *= density_unit
        self.rho_max *= density_unit
        self.mass *= mass_unit
        

In [39]:
def get_biggest_halos(num, use_density=True):
    '''
    Find the biggest halos in the simulation and return a list of Halo objects.
    
    Args
    num (int): number of halos to find
    use_density (bool): use halo density to define "biggest", rather than halo mass
    
    Returns
    list_of_halo: list of Halo objects
    '''
    biggest_halo_by_density_stdout = get_stdout("cat output*/halo* | sort -r -nk 2 | head -n 1").split()
    biggest_halo_by_density = Halo(biggest_halo_by_density_stdout)
    
    biggest_halo_by_mass_stdout = get_stdout("cat output*/halo* | sort -r -gk 7 | head -n 1").split()
    biggest_halo_by_mass = Halo(biggest_halo_by_mass_stdout)
    
    if use_density:
        list_of_stdout_split = np.array(get_stdout("cat output*/halo* | sort -r -nk 2 | head -n %d" % num).split()).reshape(-1, 7)
    else:
        list_of_stdout_split = np.array(get_stdout("cat output*/halo* | sort -r -gk 7 | head -n %d" % num).split()).reshape(-1, 7)
    
    list_of_halo = [Halo(stdout_split) for stdout_split in list_of_stdout_split]
        
    if biggest_halo_by_density.idx != biggest_halo_by_mass.idx:
        warnings.warn("Biggest density and biggest mass halo are not the same.")
    
    if num == 1:
        return list_of_halo[0]
    else:
        return list_of_halo
        
    return list_of_halo
    

In [40]:
halo = get_biggest_halos(1, use_density=False)
info_file = get_stdout("ls output*/info*").split()[0]

halo.xyz = np.array([5.105151466E-01, 5.127534424E-01, 4.929928780E-01])
    

In [41]:
ds = yt.load(info_file)


yt : [INFO     ] 2023-10-06 15:45:04,351 Parameters: current_time              = 4.006925024948247
yt : [INFO     ] 2023-10-06 15:45:04,352 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2023-10-06 15:45:04,352 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-10-06 15:45:04,353 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-10-06 15:45:04,353 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-10-06 15:45:04,353 Parameters: current_redshift          = 8.999851699314442
yt : [INFO     ] 2023-10-06 15:45:04,354 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2023-10-06 15:45:04,354 Parameters: omega_matter              = 0.275999993085861
yt : [INFO     ] 2023-10-06 15:45:04,354 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-10-06 15:45:04,354 Parameters: hubble_constant           = 0.703000030517578


In [9]:
a_exp = ds["aexp"]
H0 = ds["H0"] * const.km / const.Mpc
Omega_m0 = ds["omega_m"]
Omega_L0 = ds["omega_l"]
Omega_k0 = ds["omega_k"]
Omega_b0 = ds["omega_b"]

length_unit = ds["unit_l"]
density_unit = ds["unit_d"]
time_unit = ds["unit_t"]

max_amr_level = 13
box_size = 2 * const.kpc
left_edge = halo.coord - box_size / 2
N = int(box_size / length_unit * ds.domain_dimensions[0] * 2**amr_level)


NameError: name 'ds' is not defined

In [77]:
data = ds.covering_grid(level=amr_level, left_edge=left_edge, dims=[N]*3)


In [8]:
a_exp_to_proper_time(1.)


NameError: name 'a_exp_to_proper_time' is not defined

In [78]:
particle_type = data["io", "particle_family"].value.astype(int)
is_dm = particle_type == DM
is_star = particle_type == STAR

coord_dm = (np.array([data["io", "particle_position_x"].value[is_dm], data["io", "particle_position_y"].value[is_dm], data["io", "particle_position_z"].value[is_dm]]) - left_edge[:, None]) * length_unit - box_size / 2
mass_dm = data["io", "particle_mass"][is_dm] * mass_unit

universe_age = a_exp_to_proper_time(1.)
star_coord = (np.array([data["io", "particle_position_x"].value[is_star], data["io", "particle_position_y"].value[is_star], data["io", "particle_position_z"].value[is_star]]) - left_edge[:, None]) * length_unit - box_size / 2
star_mass = data["io", "particle_mass"][is_star] * mass_unit
star_birth_time = (data["io", "conformal_birth_time"][is_star].value / H0 + universe_age)


yt : [WARNING  ] 2023-09-27 10:11:38,011 The DM particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,012 The star particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,013 The cloud particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,013 The dust particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,014 The star_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,014 The cloud_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,014 The dust_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-09-27 10:11:38,014 The gas_tracer particle filter already exists. Overriding.
yt : [INFO     ] 2023-09-27 10:11:38,015 Adding particle_type: DM
yt : [INFO     ] 2023-09-27 10:11:38,025 Adding particle_type: star
yt : [INFO     ] 2023-09-27 10:11:38,035 Adding particle_type: cloud
yt : [INFO     ] 2

In [79]:
coord = (np.array([data["index", "x"].value, data["index", "y"].value, data["index", "z"].value]) - left_edge[:, None, None, None] - box_size / 2) * length_unit
coord1d = np.array([coord[X, :, N//2, N//2], coord[Y, N//2, :, N//2], coord[Z, N//2, N//2, :]])
density = data["ramses", "Density"].value * density_unit
metallicity = data["ramses", "Metallicity"].value 
pressure = data["ramses", "Pressure"].value * energy_density_unit
turb_energy = data["ramses", "hydro_scalar_01"].value * velocity_unit**2
refinement_criterion = data["ramses", "hydro_scalar_02"].value
vel_vec = np.array([data["ramses", "x-velocity"].value, data["ramses", "y-velocity"].value, data["ramses", "z-velocity"].value]) * velocity_unit


In [80]:
r = np.sqrt(np.sum(coord**2, axis=0))
r_dm = np.sqrt(np.sum(dm_coord**2, axis=0))
r_star = np.sqrt(np.sum(star_coord**2, axis=0))
temperature = pressure / density * const.m_H / const.k_B
vel = np.sqrt(np.sum(vel_vec**2, axis=0))


In [81]:
np.savez(
    file="gridded_data_small",
    halo_idx=halo.index,
    halo_mass=halo.mass*mass_unit,
    a_exp=a_exp,
    redshift=redshift,
    current_time=current_time,
    H0=H0,
    Omega_m0=Omega_m0,
    Omega_L0=Omega_L0,
    Omega_k0=Omega_k0,
    Omega_b0=Omega_b0,
    H=H,
    rho_crit=rho_crit,
    length_unit=length_unit,
    density_unit=density_unit,
    time_unit=time_unit,
    mass_unit=mass_unit,
    velocity_unit=velocity_unit,
    energy_unit=energy_unit,
    energy_density_unit=energy_density_unit,
    amr_level=13,
    box_size=box_size,
    left_edge=left_edge,
    N=N,
    dx=dx,
    dV=dV,
    dm_coord=dm_coord,
    dm_mass=dm_mass,
    star_coord=star_coord,
    star_mass=star_mass,
    star_birth_time=star_birth_time,
    coord=coord,
    coord1d=coord1d,
    density=density,
    metallicity=metallicity,
    pressure=pressure,
    turb_energy=turb_energy,
    refinement_criterion=refinement_criterion,
    vel_vec=vel_vec,
    r=r,
    r_dm=r_dm,
    r_star=r_star,
    temperature=temperature,
    vel=vel
)
